# Transferir el estilo de una obra textual a un modelo de escritura automatizada

Para leer el contexto en que se generó la siguiente receta leer sobre el  [Taller de escritura computarizada](https://www.notion.so/ivanvladimir/Taller-de-escritura-computarizada-e7a7bc49b552475a92b45db6416437cd)

El proceso de adaptación se lleva a cabo con los siguiente pasos
1. Instalar las librerías de python que permiten la generación
2. Conectarnos al google drive para guardar nuestro modelo y/o acceder la obra textual
3. Obtener obra textual de la que se obtendrá el estilo
4. Entrenar
5. Generar

Si ya se cuenta con un modelo entrenado, es posible brincarse el paso de entrenaiento y obtener obra (3 y 4) y pasar directamente a generar directamente

## 1. Se instalan las librerias adecuadas

Las siguentes librerias hechas por la empresa [Hugginface](https://huggingface.co/) permiten la adaptación (entrenamiento) y generación automatizada de textos. Las librerías son:
1. _transformers_, librería general para manejar modelos basados en la arquitectura neruonal transformers
2. _datasets_, librería asociada a transoformers para la manipulación de dataset/corpus de datos

Al ejecutar la celda, esperar mensajes de que se descacargan archivos y se instala módulos de python.

In [ ]:
import os
import re
!pip install transformers
!pip install datasets


## Obtener script de adaptaición (cont. 1)

La transferencia de estilo, adaptación la llevaremos a cabo usando un script que forma parte de la librería de 'transformers' para lo cual es necesario descargarlo con la siguiente celda.

Esperar unmensaje de que se descarga el archivo 'run_clm.py'

In [ ]:
!wget -O run_clm.py https://raw.githubusercontent.com/huggingface/transformers/v4.4.1/examples/language-modeling/run_clm.py

## 2. Conectar nuestro google drive

La plataforma colab puede acceder a nuestro google drive, en particular esto nos sirve para guardar el modelo adaptado en google drive y volverlo a usarlo más adelante (recordar que la máquina virtual en la que se ejecua colab deja de existir depués de un tiempo de inactividad o máximo 8 horas). 

Para lograr la conección, ejecutar la celda, y hacer click en el link generado automáticamente; confirmar con el sistema google la conexión hasta identificar el código de acceso que se pegará en la caja de texto de la misma celda. Depupués de unos segundos aparecerá el mensaje que google drive se ha 'montado' en ua ruta "/content/gdrive"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


## 3. Conseguir el texto que se utilizará para adaptar

Al proceso de adaptación hay que pasarla una obra que se adaptará, para lograr esto es necesario hacerla disponible dese la plataforma de colab, se pude hacer de tres forma:

1. Bajarlo desde el internet directamente al ambiente de Colab
2. Subirlo a colab a través del menu "Files" ubicado en la parte izquierda de la plataform
3. Usando la conexión con google drive

En general se recomienda el formato .txt en una codificiación UTF-8; también es importante tener muy claro el nombre del archivo que contiene el texto y la ruta en la que se encuentra. Si se hace a través de los pasos 3.1 y 3.2; lo más probable es que esté accesibe desde el directotio de ejecución de colab, por lo que no haría falta fijarse en la ruta.


### 3.1 Bajarlo desde internet

Usando el comando wget obtener el archivo de una URL pública con el texto. En la celda de abajo remplazar por la URL para bajar el texto recomendado. En el ejemplo se descarga El quijote en un archivo "el_quijote.txt".

Quidado, si la celda se ejecuta varias veces con el mismo nombre de archivo a bajar se crearán varias versiones del texto con números indicando la versión.

Se espera que el archivo recuperado se baje y quede visible en el directorio de ejecución de colab

In [ ]:
!wget https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

### 3.2 Subirlo a colab

Si el archivo está de forma local en la máquina utilizada para acceder a la plataforma colab, es posible subirlo a ésta. Para este proceso se usa el menu 'Files' de las izquierda siguiendo el procedimiento después de hacer click en el botón de 'Upload'. No es necesario ejecutar ninguna celda.

Se espera que el archivo subido quede visible en el directorio de ejecución de colab

### 3.3 A través de google drive

Si el archivo está en el servicio de nube de google drive, es posible acceder a ese archivo ya que tenemos conectado nuestro google drive a colab (ver paso 2). Para identificar la ruta y nombre del archivo, se puede usar el menú "Files" a la izquieda de la plataforma colab, localizar el archivo en el google drive, bajo el nombre oprimir el menu con tres punto "..." y escoger la opción "Copiar path". No es necesario ejecutar ninguna celda.

Se espera haber identificado la ruta del archivo dentro de google drive que se usará como fuente de estilo.

###3.3.1 Marcas en el texto
Para mantener la estructura del texto es recomendable indicar la longitud de cada línea y de cada párrafo, por ello la siguiente celda recupera un archivo de un ruta dentro del Google Drive personal y anexa una codificación en cada línea y en cada párrafo de dicho archivo, el modelo es entrenado con dicha codificación, la aprende y genera un texto con la estructura aprendida pero decodificada nuevamente al final de esta notebook


In [ ]:
contenido= list()

with open('/content/gdrive/MyDrive/Textos-TallerCCD/constitucion.txt', 'r',  encoding='utf-8', errors='ignore') as archivo:
       for linea in archivo:
            if not linea == '\n':
                linea.rstrip('\n')
                linea2 = ' -- '+linea+' -- \n'
            else:
                linea2 = '\n'
            
            #print(linea2)
            l = linea2.split('\n --')
            #print(l)
            
            if l == ['\n']:
                l = '*p'.join(l)
                #print(p)
                contenido.append('*p \n'+ l + '*p')
                
            elif not l == ['\n'] :
                l = ' -- '.join(l)
                contenido.append((l))
            
        

with open('/content/gdrive/MyDrive/Textos-TallerCCD/constitucion_marcas.txt', 'w',  encoding='utf-8', errors='ignore') as archivo:
    archivo.writelines(contenido)

## 4. Entrenamiento para adaptar el modelo

La siguiente celda es la que adapta un modelo pre entrenado 'datificate/gpt2-small-spanish' con el estilo del texto fuente.

En este paso será necesario tener claro el nombre y la ruta (para paso 3.3) del archivo textual que se usará como fuente para el estilo (ver paso 3). Esa ruta (de ser necesaria) y nombre se sustituira en la celda de abajo para la opción '--train_file' 

Cuidado en el formato de la celda es muy imporante mantener el símbolo '\' al final de la línea en la opción '--train_file' y que no haya ningún símbolo o espacio después de éste.

Al ejecutar la siguiente línea puede llevarse varios minutos o hasta horas

In [ ]:
!mkdir output
!python run_clm.py \
    --model_name_or_path 'datificate/gpt2-small-spanish' \
    --train_file '/content/gdrive/MyDrive/Textos-TallerCCD/constitucion_marcas.txt'\
    --do_train \
    --block_size 128\
    --num_train_epochs 40 \
    --overwrite_output_dir \
    --save_total_limit 0 \
    --output_dir /output/gpt2-small-spanish-adaptado

## 4.2 Copiar modelo a google drive

La siguiente celda copia el modelo a una carpta dentro de google drive llamada "models" si se quiere poner en otra carpeta adaptar la celda.

In [ ]:
# Moviendo el modelo al google drive
output_dir = '/content/drive/MyDrive/models/'
!mkdir -p '/content/drive/MyDrive/models/'
!cp -r '/output/gpt2-small-spanish-adaptado' '{output_dir}'
!ls '{output_dir}/gpt2-small-spanish-adaptado'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## 4.3 Más opciones al script

La siguiente celda muestra todas las opciones diponibles en el texto, por si se requiere hacer de modificaciones al entrenamiento

In [ ]:
!python run_clm.py -h

## 5. Generando texto

La sigueintes celdas generan el texto, la primera recupera el modelo entrenado desde nuestro google drive, y la segunda hace la ejecuión. Uno puede ejecutar la segunda opción tantas veces como vea uno necesario. La opción _max_length_ controla la cantidad de texto generado.

In [ ]:
from transformers import pipeline
model = "/content/drive/MyDrive/models/gpt2-small-spanish-adaptado"
model_text = pipeline('text-generation',model=model, tokenizer=model,)

In [ ]:
poem=model_text("Asi podemos comenzar",max_length=500)[0]['generated_text']

In [ ]:
poem_=re.sub(r"--\s*?--","\n",poem,flags=re.DOTALL)
poem_=re.sub(r"\n\s+","\n",poem_,flags=re.DOTALL)
poem_=re.sub(r"\*p","\n",poem_,flags=re.DOTALL)
poem_=re.sub(r"\*P -- (.*?)\s+\n","\g<1>\n\n",poem_,flags=re.DOTALL)
poem_=re.sub(r"-- ","\n",poem_,flags=re.DOTALL)
print(poem_)